<a href="https://colab.research.google.com/github/priyanka0178885547/ML-LAB/blob/main/week4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df=pd.read_csv('/content/titanic_train.csv')
df.shape

(891, 12)

In [ ]:
df.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [ ]:
df=df.drop(['PassengerId','Name','Ticket'],axis=1)

In [ ]:
df.head(5)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,0,3,male,22.0,1,0,7.2500,NaN,S
1,1,1,female,38.0,1,0,71.2833,C85,C
2,1,3,female,26.0,0,0,7.9250,NaN,S
3,1,1,female,35.0,1,0,53.1000,C123,S
4,0,3,male,35.0,0,0,8.0500,NaN,S


In [ ]:
[col for col in df.columns if df[col].isnull().sum()>0]


['Age', 'Cabin', 'Embarked']

In [ ]:
df['Age'] = df['Age'].fillna(df['Age'].median())

In [ ]:
df=df.drop(['Embarked','Cabin'],axis=1)

In [ ]:
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor

In [ ]:
from sklearn.metrics import roc_auc_score

In [ ]:
from mlxtend.feature_selection import SequentialFeatureSelector as SFS

In [ ]:
X = df.drop('Survived',axis=1)
y = df['Survived']

In [ ]:
X = pd.get_dummies(X,drop_first=True)


In [ ]:
from sklearn.model_selection import train_test_split


In [ ]:
x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=0)

In [ ]:
grouped_feature_ls = []
correlated_groups = []
def correlation (dataset, threshold):
    col_corr = set()
    corr_matrix = dataset.corr()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if abs(corr_matrix.iloc[i, j]) > threshold:
                colname=corr_matrix.columns[i]
                col_corr.add(colname)
    return col_corr
corr_features=correlation (x_test, 0.9)
correlated_features = set(corr_features)
print(len(correlated_features))

0


In [ ]:
sfs1=SFS(RandomForestClassifier(n_jobs=4),k_features=5,forward=True,verbose=2,scoring='roc_auc' or 'accuracy',cv=3)


In [ ]:
sfs1=sfs1.fit(np.array(x_train.fillna(0)),y_train)

[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:    5.0s finished

[2025-08-20 09:37:15] Features: 1/5 -- score: 0.765194136612553[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    4.6s finished

[2025-08-20 09:37:19] Features: 2/5 -- score: 0.832223720877483[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    3.5s finished

[2025-08-20 09:37:23] Features: 3/5 -- score: 0.8358966413920483[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    2.7s finished

[2025-08-20 09:37:26] Features: 4/5 -- score: 0.8412565616797901[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    1.7s finished

[2025-08-20 09:37:27] Features: 5/5 -- score: 0.8447287839020122

In [ ]:
print('best accracy score',sfs1.k_score_)
print('best subset(indices)',sfs1.k_feature_idx_)
print('Best subset (corresponding names):', sfs1.k_feature_names_)

best accracy score 0.8447287839020122
best subset(indices) (0, 1, 2, 4, 5)
Best subset (corresponding names): ('0', '1', '2', '4', '5')


In [ ]:
sfs2=SFS(RandomForestClassifier(n_jobs=4),k_features=5,forward=False,verbose=2,scoring='roc_auc' or 'accuracy',cv=3)
sfs2=sfs2.fit(np.array(x_train.fillna(0)),y_train)
print('best accracy score',sfs2.k_score_)
print('best subset(indices)',sfs2.k_feature_idx_)
print('Best subset (corresponding names):', sfs2.k_feature_names_)

best accracy score 0.8428028093710509
best subset(indices) (0, 1, 2, 4, 5)
Best subset (corresponding names): ('0', '1', '2', '4', '5')


[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:    6.3s finished

[2025-08-20 09:37:44] Features: 5/5 -- score: 0.8428028093710509

In [ ]:
from mlxtend.feature_selection import ExhaustiveFeatureSelector as EFS
efs=EFS(RandomForestClassifier(n_jobs=4),min_features=1,max_features=4,scoring='accuracy',cv=5)
efs=efs.fit(x_train,y_train)
print('best accuracy score',efs.best_score_)
print('best subset(indices)',efs.best_idx_)
print('Best subset (corresponding names):', efs.best_feature_names_)

Features: 56/56

best accuracy score 0.8170580645161291
best subset(indices) (0, 1, 5)
Best subset (corresponding names): ('Pclass', 'Age', 'Sex_male')


In [ ]:
efs_results = pd.DataFrame.from_dict(efs.get_metric_dict()).T
efs_results.sort_values(by='avg_score', ascending=True, inplace=True)
efs_results

,feature_idx,cv_scores,avg_score,feature_names,ci_bound,std_dev,std_err
11,"(1, 2)","[0.656, 0.576, 0.624, 0.5564516129032258, 0.58...",0.600232,"(Age, SibSp)",0.045666,0.035529,0.017765
12,"(1, 3)","[0.56, 0.568, 0.656, 0.6290322580645161, 0.596...",0.601961,"(Age, Parch)",0.046676,0.036315,0.018158
34,"(1, 3, 4)","[0.576, 0.6, 0.576, 0.6048387096774194, 0.6532...",0.602013,"(Age, Parch, Fare)",0.0363,0.028242,0.014121
1,"(1,)","[0.624, 0.592, 0.624, 0.5887096774193549, 0.58...",0.603484,"(Age,)",0.021586,0.016794,0.008397
13,"(1, 4)","[0.592, 0.64, 0.592, 0.5564516129032258, 0.637...",0.60351,"(Age, Fare)",0.040396,0.031429,0.015715
3,"(3,)","[0.568, 0.6, 0.624, 0.6048387096774194, 0.6451...",0.6084,"(Parch,)",0.03307,0.02573,0.012865
31,"(1, 2, 3)","[0.64, 0.576, 0.656, 0.6209677419354839, 0.629...",0.6244,"(Age, SibSp, Parch)",0.034573,0.026899,0.013449
2,"(2,)","[0.624, 0.6, 0.656, 0.5645161290322581, 0.7177...",0.632452,"(SibSp,)",0.066955,0.052094,0.026047
23,"(0, 1, 4)","[0.632, 0.648, 0.64, 0.6048387096774194, 0.669...",0.638839,"(Pclass, Age, Fare)",0.027078,0.021068,0.010534
21,"(0, 1, 2)","[0.608, 0.624, 0.712, 0.6048387096774194, 0.65...",0.640413,"(Pclass, Age, SibSp)",0.051006,0.039685,0.019842


In [ ]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(random_state=0, criterion='gini')
rfc.fit(x_train, y_train)
for feature in zip(x_train.columns, rfc.feature_importances_):
    print(feature)


('Pclass', np.float64(0.08580548090109545))
('Age', np.float64(0.26433757901109173))
('SibSp', np.float64(0.05355992352286061))
('Parch', np.float64(0.03699908167792097))
('Fare', np.float64(0.28790097031129963))
('Sex_male', np.float64(0.2713969645757317))


In [ ]:
from sklearn.feature_selection import SelectFromModel
sfm = SelectFromModel(estimator=rfc, threshold=0.2)
sfm = sfm.fit(x_train, y_train)
feature_names=x_train.columns
print('The most important features based on random forest classifier:')
for feature_list_index in sfm.get_support(indices=True):
    print('- ' + feature_names[feature_list_index])

The most important features based on random forest classifier:
- Age
- Fare
- Sex_male


In [ ]:
from sklearn.metrics import accuracy_score
rfc_full = RandomForestClassifier(random_state=0, criterion='gini')
rfc_full.fit(x_train, y_train)
pred_full = rfc_full.predict(x_test)
print('The accuracy of classifier with full features: {:.2f}'.format(accuracy_score(y_test, pred_full)))


The accuracy of classifier with full features: 0.82


In [ ]:
x_important_train = sfm.transform(x_train)
x_important_test = sfm.transform(x_test)

In [ ]:
rfc_lim = RandomForestClassifier(random_state=0, criterion='gini')
rfc_lim.fit(x_important_train, y_train)
pred_lim = rfc_lim.predict(x_important_test)
print('The accuracy of classifier with limited features: {:.2f}'.format(accuracy_score(y_test, pred_lim)))

The accuracy of classifier with limited features: 0.80


In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df1=pd.read_csv('/content/titanic_train.csv')
df1.shape

(891, 12)

In [ ]:
[col for col in df1.columns if df1[col].isnull().sum()>0]

['Age', 'Cabin', 'Embarked']

In [ ]:
df1['Age'] = df['Age'].fillna(df1['Age'].median())

In [ ]:
df1=df1.drop(['Embarked','Cabin'],axis=1)

In [ ]:
X = df.drop('Survived',axis=1)
y = df['Survived']

In [ ]:
X = pd.get_dummies(X,drop_first=True)

In [ ]:

data = pd.concat([X, y], axis=1)
data = data.dropna()
X = data.drop(columns=['Survived'])
y = data['Survived']


In [ ]:
scaler=StandardScaler()
x_scaled=scaler.fit_transform(x_train)

In [ ]:
x_scaled.shape

(623, 6)

In [ ]:
pca=PCA(n_components=2)
x_pca=pca.fit_transform(x_scaled)

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x_pca,y,test_size=0.3,random_state=42)
model=LogisticRegression()
model.fit(x_train,y_train)
y_pred=model.predict(x_test)

In [ ]:
cm=confusion_matrix(y_test,y_pred)
plt.figure(figsize=(8,6))
sns.heatmap(cm,annot=True,fmt='d',cmap='Blues')
plt.title('Confusion Matrix')
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.show()

In [ ]:
y_numeric=pd.actegorize(y)[0]
plt.figure(figsize=(12,5))
plt.subplot(1,2,1)
plt.scatter(x_scaled[:,0],x_scaled[:,1],c=y_numeric,cmap='viridis',edgecolor='k',s=50
sns.scatter(x_scaled[:0]),x_scaled[:,1],c=y_numeric,cmap='viridis',edgecolor='k',s=50)
plt.xlabel('Original Feature 1')
plt.ylabel('Original FEature 2')
plt.title("Before PCA:using first two standardized features")
plt.colorbar(label="Target classes")
plt.subplot(1,2,2)
plt.scatter(x_pca[:,0],x_pca[:,1],c=y_numeric,cmap='viridis',edgecolor='k',s=50)
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.title("After PCA:using first two principal components")
plt.colorbar(label="Target classes")
plt.tight_layout()
plt.show()